# Common statistical tests are linear models (or: how to teach stats)

[Source](https://lindeloev.github.io/tests-as-linear/)

This document is summarised in the table below. It shows the linear models underlying common parametric and “non-parametric” tests. Formulating all the tests in the same language highlights the many similarities between them. 

![title](images/r_linear_tests_cheat_sheet.png)

## 1 The simplicity underlying common tests

Most of the common statistical models (t-test, correlation, ANOVA; chi-square, etc.) are special cases of linear models or a very close approximation. This beautiful simplicity means that there is less to learn. In particular, it all comes down to $ y=a⋅x+b $ which most students know from high school. Unfortunately, stats intro courses are usually taught as if each test is an independent tool, needlessly making life more complicated for students and teachers alike.

This needless complexity multiplies when students try to rote learn the parametric assumptions underlying each test separately rather than deducing them from the linear model.

For this reason, I think that teaching linear models first and foremost and then name-dropping the special cases along the way makes for an excellent teaching strategy, emphasizing understanding over rote learning. Since linear models are the same across frequentist, Bayesian, and permutation-based inferences, I’d argue that it’s better to start with modeling than p-values, type-1 errors, Bayes factors, or other inferences.

Use the menu to jump to your favourite section. There are links to lots of similar (though more scattered) stuff under sources and teaching materials. I hope that you will join in suggesting improvements or submitting improvements yourself in the Github repo to this page. Let’s make it awesome!

## 2 Settings and Toy Data

In [ ]:
# Load packages for data handling and plotting
library(tidyverse)
library(patchwork)
library(broom)

In [ ]:
# Reproducible "random" results
set.seed(40)

# Generate normal data with known parameters
rnorm_fixed = function(N, mu = 0, sd = 1) {
  scale(rnorm(N)) * sd + mu
}

# Plot style.
theme_axis = function(P,
                      jitter = FALSE,
                      xlim = c(-0.5, 2),
                      ylim = c(-0.5, 2),
                      legend.position = NULL) {
  P = P + theme_bw(15) +
    geom_segment(
      x = -1000, xend = 1000,
      y = 0, yend = 0,
      lty = 2, color = 'dark gray', lwd = 0.5
    ) +
    geom_segment(
      x = 0, xend = 0,
      y = -1000, yend = 1000,
      lty = 2, color = 'dark gray', lwd = 0.5
    ) +
    coord_cartesian(xlim = xlim, ylim = ylim) +
    theme(
      axis.title = element_blank(),
      axis.text = element_blank(),
      axis.ticks = element_blank(),
      panel.border = element_blank(),
      panel.grid = element_blank(),
      legend.position = legend.position
    )
  
  # Return jittered or non-jittered plot?
  if (jitter) {
    P + geom_jitter(width = 0.1, size = 2)
  }
  else {
    P + geom_point(size = 2)
  }
}

For a start, we’ll keep it simple and play with three standard normals in wide (`a`, `b`, `c`) and long format (`value`, `group`):

In [ ]:
# Wide format (sort of)
#y = rnorm_fixed(50, mu=0.3, sd=2)  # Almost zero mean.
y = c(rnorm(15), exp(rnorm(15)), runif(20, min = -3, max = 0))  # Almost zero mean, not normal
x = rnorm_fixed(50, mu = 0, sd = 1)  # Used in correlation where this is on x-axis
y2 = rnorm_fixed(50, mu = 0.5, sd = 1.5)  # Used in two means

# Long format data with indicator
value = c(y, y2)
group = rep(c('y1', 'y2'), each = 50)

## 3 Pearson and Spearman Correlation

#### 3.0.1 Theory: As linear models ####

Model: the recipe for $y$ is a slope ($\beta_1$) times $x$ plus an intercept ($\beta_0$, aka a straight line).

$$ y = \beta_0 + \beta_1x $$  $$ H_0 : \beta_1 = 0 $$

This is a mathy way of writing the good old $y = mx + b$ (but here reordered as $y = b + mx$). In R we are lazy and write `y ~ 1 + x` which R reads like `y = 1 * number + x * othernumber` and the task of t-tests, lm, etc is simply to find numbers that best predict $y$.

However you write it, it's an intercept ($\beta_0$) and a slope ($\beta_1$) yielding a straight line.

In [ ]:
# Created fixed correlation data
D_correlation = data.frame(MASS::mvrnorm(30, mu = c(0.9, 0.9), Sigma = matrix(c(1, 0.8, 1, 0.8), ncol = 2), empirical = TRUE))

# Add labels (for next plot)
D_correlation$label_num = sprintf('(%.1f,%.1f)', D_correlation$X1, D_correlation$X2)
D_correlation$label_rank = sprintf('(%i,%i)', rank(D_correlation$X1), rank(D_correlation$X2))

# Plot it
fit = lm(I(X2 * 0.5 + 0.4) ~ I(X1 * 0.5 + 0.2), D_correlation)
intercept_pearson = coefficients(fit)[1]

P_pearson = ggplot(D_correlation, aes(x=X1*0.5+0.2, y=X2*0.5+0.4)) +
  geom_smooth(method=lm, se=FALSE, lwd=2, aes(colour='beta_1')) + 
  geom_segment(x=-100, xend=100, 
               y=intercept_pearson, yend=intercept_pearson, 
               lwd=2, aes(color="beta_0")) + 
  scale_color_manual(name=NULL, values=c("blue", "red"), labels=c(bquote(beta[0]*" (intercept)"), bquote(beta[1]*" (slope)")))
  
theme_axis(P_pearson, legend.position = c(0.4, 0.9))

This is often simply called a **regression** model which can be extended to multiple regression where there are several $\beta$s and on the right-hand side multiplied with the predictors. **Everything below, from one-sample t-test to two-way ANOVA are just special cases of this system. Nothing more, nothing less.**

As the name implies, the Spearman rank correlation is a Pearson correlation on rank-transformed $x$ and $y$:

$$ rank(y) = \beta_0 + \beta_1 \cdot rank(x) $$ $$ H_0 : \beta_1 = 0 $$

I’ll introduce ranks in a minute. For now, notice that the correlation coefficient of the linear model is identical to a “real” Pearson correlation, but p-values are an approximation which is is appropriate for samples greater than N=10 and almost perfect when N > 20.

Such a nice and non-mysterious equivalence that many students are left unaware of! Visualizing them side by side including data labels, we see this rank-transformation in action:

In [ ]:
# Spearman intercept
intercept_spearman = coefficients(lm(rank(X2) ~ rank(X1), D_correlation))[1]

# Spearman plot
P_spearman = ggplot(D_correlation, aes(x=rank(X1), y=rank(X2))) +
  geom_smooth(method=lm, se=FALSE, lwd=2, aes(color='beta_1')) + 
  geom_text(aes(label=label_rank), nudge_y=1, size=3, color='dark gray') + 
  geom_segment(x=-100, xend=100, 
               y=intercept_spearman, yend=intercept_spearman, 
               lwd=2, aes(color='beta_0')) + 
  scale_color_manual(name=NULL, values=c("blue", "red"), labels=c(bquote(beta[0]*" (intercept)"), bquote(beta[1]*" (slope)")))

# Stich together using patchwork
(theme_axis(P_pearson, legend.position=c(0.5, 0.1)) + geom_text(aes(label=label_num), nudge_y=0.1, size=3, color='dark gray') + labs(title='         Pearson')) + (theme_axis(P_spearman, xlim=c(-7.5, 30), ylim=c(-7.5, 30), legend.position=c(0.5, 0.1)) + labs(title='         Spearman'))

#### 3.0.2 Theory: Rank-transformation ####

`rank` simply takes a list of numbers and “replaces” them with the integers of their rank (1st smallest, 2nd smallest, 3rd smallest, etc.). So the result of the rank-transformation `rank(c(3.6, 3.4, -5.0, 8.2))` is `3, 2, 1, 4`. See that in the figure above?

A *signed* rank is the same, just where we rank according to absolute size first and then add in the sign second. So the signed rank here would be 2, 1, -3, 4. Or in code:

In [ ]:
signed_rank = function(x) sign(x) * rank(abs(x))

I hope I don’t offend anyone when I say that ranks are easy; yet it’s all you need to do to convert most parametric tests into their “non-parametric” counterparts! *One interesting implication is that many “non-parametric tests” are about as parametric as their parametric counterparts with means, standard deviations, homogeneity of variance, etc. - just on rank-transformed data. That’s why I put “non-parametric” in quotation marks.*

#### 3.0.3 R code: Pearson correlation ####

It couldn’t be much simpler to run these models in R. They yield identical `p` and `t`, but there’s a catch: `lm` gives you the *slope* and even though that is usually much more interpretable and informative than the correlation coefficient `r`, you may still want `r`. Luckily, the **slope becomes `r`** if `x` and `y` have identical standard deviations. For now, we will use `scale(x)` to make $SD(x)=1.0$ and $SD(y)=1.0$. The CIs are not exactly identical, but very close.

In [ ]:
a = cor.test(y, x, method = "pearson") # Built-in
b = lm(y ~ 1 + x) # Equivalent linear model: y = Beta0*1 + Beta1*x
c = lm(scale(y) ~ 1 + scale(x))  # On scaled vars to recover r

In [ ]:
a

In [ ]:
summary(b)

In [ ]:
stats::confint(b, level = 0.95)

In [ ]:
summary(c)

In [ ]:
stats::confint(c, level = 0.95)

### 3.0.4 R code: Spearman correlation ###

Note that we can interpret the slope which is the number of ranks $y$ change for each rank on $x$. I think that this is a pretty interesting number. However, the intercept is less interpretable since it lies at $rank(x)=0$ which is impossible since x starts at 1.

See the identical `r` (now “rho”) and `p`:

In [ ]:
# Spearman correlation
a = cor.test(y, x, method = "spearman") # Built-in
b = lm(rank(y) ~ 1 + rank(x)) # Equivalent linear model

In [ ]:
a

In [ ]:
summary(b)

## 4 One Mean
### 4.1 One Sample t-test and Wilcoxon signed-rank
#### 4.1.1 Theory: As Linear Models
**t-test model**: A single number predicts $y$.
$$ y = \beta_0$$ $$ H_0 : \beta_0 = 0 $$
In other words, it’s our good old $ y=\beta_0+\beta_1∗x $ where the last term is gone since there is no $x$ (essentially $ x=0 $, see left figure below).

The same is approximately true for Wilcoxon signed-rank test, just with the signed ranks of $y$ instead of $y$ itself (see right panel below).

signed_rank(y)=β0

In [ ]:
# T-test
D_t1 = data.frame(y = rnorm_fixed(20, 0.5, 0.6),
                  x = runif(20, 0.93, 1.07))  # Fix mean and SD

P_t1 = ggplot(D_t1, aes(y = y, x = 0)) + 
  stat_summary(fun.y=mean, geom = "errorbar", aes(ymax = ..y.., ymin = ..y.., color='beta_0'), lwd=2) +
  scale_color_manual(name = NULL, values = c("blue"), labels = c(bquote(beta[0] * " (intercept)"))) +
  
  geom_text(aes(label = round(y, 1)), nudge_x = 0.2, size = 3, color = 'dark gray') + 
  labs(title='         T-test')

# Wilcoxon
D_t1_rank = data.frame(y = signed_rank(D_t1$y))

P_t1_rank = ggplot(D_t1_rank, aes(y = y, x = 0)) + 
  stat_summary(fun.y = mean, geom = "errorbar", aes(ymax = ..y.., ymin = ..y..,  color = 'beta_0'), lwd = 2) +
  scale_color_manual(name = NULL, values = c("blue"), labels = c(bquote(beta[0] * " (intercept)"))) +

  geom_text(aes(label = y), nudge_x = 0.2, size = 3, color = 'dark gray') + 
  labs(title='         Wilcoxon')


# Stich together using patchwork
theme_axis(P_t1, ylim = c(-1, 2), legend.position = c(0.6, 0.1)) + 
  theme_axis(P_t1_rank, ylim = NULL,  legend.position = c(0.6, 0.1))

#### 4.1.2 R code: One-sample t-test

Try running the R code below and see that the linear model (lm) produces the same $t$, $p$, and $r$ as the built-in t.test. The confidence interval is not presented in the output of lm but is also identical if you use `stats::confint(lm(...))`:

In [ ]:
# Built-in t-test
a = t.test(y)

# Equivalent linear model: intercept-only
b = lm(y ~ 1)

In [ ]:
a

In [ ]:
summary(b)

In [ ]:
stats::confint(b)

#### 4.1.3 R code: Wilcoxon signed-rank test

In addition to matching p-values, `lm` also gives us the mean signed rank, which I find to be an informative number.

In [ ]:
# Built-in
a = wilcox.test(y)

# Equivalent linear model
b = lm(signed_rank(y) ~ 1)  # See? Same model as above, just on signed ranks

# Bonus: of course also works for one-sample t-test
c = t.test(signed_rank(y))

In [ ]:
a

In [ ]:
summary(b)

In [ ]:
c

### 4.2 Paired samples t-test and Wilcoxon matched pairs
#### 4.2.1 Theory: As linear models

**t-test model:** a single number (intercept) predicts the pairwise differences.
$$ y_2 - y_1 = \beta_0 $$  $$ H_0 : \beta_0 = 0 $$
This means that there is just one $y=y2−y1$ to predict and it becomes a one-sample t-test on the pairwise differences. The visualization is therefore also the same as for the one-sample t-test. At the risk of overcomplicating a simple substraction, you can think of these pairwise differences as slopes (see left panel of the figure), which we can represent as y-offsets (see right panel of the figure):

In [ ]:
# Data for plot
N = nrow(D_t1)
start = rnorm_fixed(N, 0.2, 0.3)
D_tpaired = data.frame(
  x = rep(c(0, 1), each = N),
  y = c(start, start + D_t1$y),
  id = 1:N
)

# Plot
P_tpaired = ggplot(D_tpaired, aes(x = x, y = y)) +
  geom_line(aes(group = id)) +
  labs(title = '          Pairs')

# Use patchwork to put them side-by-side
theme_axis(P_tpaired) + theme_axis(P_t1, legend.position = c(0.6, 0.1))

Similarly, the **Wilcoxon matched pairs** only differ from **Wilcoxon signed-rank** in that it’s testing the signed ranks of the pairwise $y−x$ differences.

$$signed\_rank(y_2−y_1)=\beta_0$$ $$H_0:\beta_0=0$$

#### 4.2.2 R Code: Paired Sample t-test

In [ ]:
a = t.test(y, y2, paired = TRUE) # Built-in paired t-test
b = lm(y - y2 ~ 1) # Equivalent linear model

In [ ]:
a

In [ ]:
summary(b)

In [ ]:
stats::confint(b)

In [ ]:
# Built-in Wilcoxon matched pairs
a = wilcox.test(y, y2, paired = TRUE)

# Equivalent linear model:
b = lm(signed_rank(y - y2) ~ 1)

# Bonus: identical to one-sample t-test ong signed ranks
c = t.test(signed_rank(y - y2))

In [ ]:
a

In [ ]:
summary(b)

In [ ]:
c

## 5 Two means
### 5.1 Independent t-test and Mann-Whitney U
#### 5.1.1 Theory: As linear models

**Independent t-test model**: two means predict $y$

$$ y_i = \beta_0 + \beta_1x_i $$ $$ H_0 : \beta_1 = 0 $$

where $x_i$ is an indicator (0 or 1) saying whether data point $i$ was sampled from one or the other group. Indicator variables (also called “[dummy coding](https://en.wikipedia.org/wiki/Dummy_variable_(statistics))" underly a lot of linear models and we’ll take an aside to see how it works in a minute.

**Mann-Whitney U** (also known as **Wilcoxon rank-sum test** for two independent groups; no signed rank this time) is the same model to a very close approximation, just on the *ranks* of $x$ and $y$ instead of the actual values:

$$ rank(y_i)=\beta_0+\beta_1x_i$$ $$H_0:\beta_1=0$$

To me, equivalences like this make “non-parametric” statistics much easier to understand. The approximation is appropriate when the sample size is larger than 11 in each group and virtually perfect when N > 30 in each group.

#### 5.1.2 Theory: Dummy coding

Dummy coding can be understood visually. The indicator is on the x-axis so data points from the first group are located at $x=0$ and data points from the second group is located at $x=1$. Then $\beta_0$ is the intercept (blue line) and $\beta_1$ is the slope between the two means (red line). Why? Because when $\Delta_x=1$ the slope equals the difference because:

$$ slope=\Delta_y/\Delta_x=\Delta_y/1=\Delta_y=difference $$

Magic! Even categorical differences can be modelled using linear models! It’s a true Swiss army knife.

In [ ]:
# Data
N = 20  # Number of data points per group
D_t2 = data.frame(
  x = rep(c(0, 1), each=N),
  y = c(rnorm_fixed(N, 0.3, 0.3), rnorm_fixed(N, 1.3, 0.3))
)

# Plot
P_t2 = ggplot(D_t2, aes(x=x, y=y)) + 
  stat_summary(fun.y = mean, geom = "errorbar", aes(ymax = ..y.., ymin = ..y..,  color = 'something'), lwd = 2) +
  geom_segment(x = -10, xend = 10, y = 0.3, yend = 0.3, lwd = 2, aes(color = 'beta_0')) + 
  geom_segment(x = 0, xend = 1, y = 0.3, yend = 1.3, lwd = 2, aes(color = 'beta_1')) + 
  
  scale_color_manual(name = NULL, values = c("blue", "red", "darkblue"), labels=c(bquote(beta[0]*" (group 1 mean)"), bquote(beta[1]*" (slope = difference)"), bquote(beta[0]+beta[1]%.%1*" (group 2 mean)")))
  #scale_x_discrete(breaks=c(0.5, 1.5), labels=c('1', '2'))

theme_axis(P_t2, jitter = TRUE, xlim = c(-0.3, 2), legend.position = c(0.53, 0.08))

#### 5.1.3 Theory: Dummy coding (continued)

If you feel like you get dummy coding now, just skip ahead to the next section. Here is a more elaborate explanation of dummy coding:

If a data point was sampled from the first group, i.e., when $x_i=0$, the model simply becomes $y=\beta_0+\beta_1\cdot0=\beta_0$ . In other words, the model predicts that that data point is $\beta_0$. It turns out that the $\beta$ which best predicts a set of data points is the mean of those data points, so $\beta_0$ is the mean of group 1.

On the other hand, data points sampled from the second group would have $x_i=1$ so the model becomes $y_i=\beta_0+\beta_1⋅1 = \beta_0+\beta_1$. In other words, we add $\beta_1$ to “shift” from the mean of the first group to the mean of the second group. Thus $\beta_1$ becomes the mean difference between the groups.

As an example, say group 1 is 25 years old ($\beta_0=25$) and group 2 is 28 years old ($\beta_1=3$), then the model for a person in group 1 is $y=25+3⋅0=25$ and the model for a person in group 2 is $y=25+3⋅1=28$.

Hooray, it works! For first-timers it takes a few moments to understand dummy coding, but you only need to know addition and multiplication to get there!

#### R code: Independent t-test

As a reminder, when we write `y ~ 1 + x` in R, it is shorthand for $y=\beta_0⋅1+\beta_1⋅x$ and R goes on computing the $\beta$s for you. Thus `y ~ 1 + x` is the R-way of writing $y=a⋅x+b$.

Notice the identical `t`, `df`, `p`, and estimates. We can get the confidence interval by running `stats::confint(lm(...))`.

In [ ]:
# Built-in independent t-test on wide data
a = t.test(y, y2, var.equal = TRUE)

# Be explicit about the underlying linear model by hand-dummy-coding:
group_y2 = ifelse(group == 'y2', 1, 0)  # 1 if group == y2, 0 otherwise
b = lm(value ~ 1 + group_y2)  # Using our hand-made dummy regressor

# Note: We could also do the dummy-coding in the model
# specification itself. Same result.
c = lm(value ~ 1 + I(group == 'y2'))

In [ ]:
a

In [ ]:
summary(b)

**Note on above**: `mean_x`, or mean of group y1 equals the intercept coefficient. `mean_y`, or mean of group y2 equals group_y2 coefficient minus intercept coefficient.

In [ ]:
stats::confint(b)

In [ ]:
# Wilcoxon / Mann-Whitney U
a = wilcox.test(y, y2)

# As linear model with our dummy-coded group_y2:
b = lm(rank(value) ~ 1 + group_y2)  # compare to linear model above

In [ ]:
a

In [ ]:
summary(b)

### Welch's t-test

This is identical to the (Student’s) independent t-test above except that Student’s assumes identical variances and Welch’s t-test does not. So the linear model is the same but we model one variance per group. We can do this using the nlme package ([see more details here](https://stats.stackexchange.com/questions/142685/equivalent-to-welchs-t-test-in-gls-framework)):

In [ ]:
# Built-in
a = t.test(y, y2, var.equal=FALSE)

# As linear model with per-group variances
b = nlme::gls(value ~ 1 + group_y2, weights = nlme::varIdent(form=~1|group), method="ML")

In [ ]:
a

In [ ]:
summary(b)

In [ ]:
stats::confint(b)

## 6 Three or more means

ANOVAs are linear models with (only) categorical predictors so they simply extend everything we did above, relying heavily on dummy coding. Do make sure to read the section on dummy coding if you haven't already.

### 6.1 One-way ANOVA and Kruskal-Wallis
#### 6.1.1 Theory: As linear models

Model: One mean for each group predicts $y$.

$$ y = \beta_0 + \beta_1x_1 + \beta_2x_2 +\beta_3x_3 + ... $$ $$ H_0: y = \beta_0$$

where $x_i$ are indicators ($x=0$ or $x=1$) where at most one $x_i=1$ while all others are $x_i=0$. 

Notice how this is just “more of the same” of what we already did in other models above. When there are only two groups, this model is $y=\beta_0+\beta_1∗x$, i.e. **the independent t-test**. If there is only one group, it is $y=\beta_0$, i.e. **the one-sample t-test**. This is easy to see in the visualization below - just cover up a few groups and see that it matches the other visualizations above.

In [ ]:
# Figure
N = 15
D_anova1 = data.frame(
  y = c(
    rnorm_fixed(N, 0.5, 0.3),
    rnorm_fixed(N, 0, 0.3),
    rnorm_fixed(N, 1, 0.3),
    rnorm_fixed(N, 0.8, 0.3)
  ),
  x = rep(0:3, each = 15)
)
ymeans = aggregate(y~x, D_anova1, mean)$y
P_anova1 = ggplot(D_anova1, aes(x=x, y=y)) + 
  stat_summary(fun.y=mean, geom = "errorbar", aes(ymax = ..y.., ymin = ..y.., color='intercepts'), lwd=2) + 
  geom_segment(x = -10, xend = 100, y = 0.5, yend = 0.5, lwd = 2, aes(color = 'beta_0')) +
  geom_segment(x = 0, xend = 1, y = ymeans[1], yend = ymeans[2], lwd = 2, aes(color = 'betas')) +
  geom_segment(x = 1, xend = 2, y = ymeans[1], yend = ymeans[3], lwd = 2, aes(color = 'betas')) +
  geom_segment(x = 2, xend = 3, y = ymeans[1], yend = ymeans[4], lwd = 2, aes(color = 'betas')) +
  
  scale_color_manual(
    name = NULL, values = c("blue", "red", "darkblue"), 
    labels=c(
      bquote(beta[0]*" (group 1 mean)"), 
      bquote(beta[1]*", "*beta[2]*",  etc. (slopes/differences to "*beta[0]*")"),
      bquote(beta[0]*"+"*beta[1]*", "*beta[0]*"+"*beta[2]*", etc. (group 2, 3, ... means)")
    )
  )
  

theme_axis(P_anova1, xlim = c(-0.5, 4), legend.position = c(0.7, 0.1))

A one-way ANOVA has a log-linear counterpart called **goodness-of-fit** test which we’ll return to. By the way, since we now regress on more than one $x$, the one-way ANOVA is a **multiple regression model**.

The Kruskal-Wallis test is simply a one-way ANOVA on the rank-transformed $y$(`value`):

$$rank(y)=\beta_0+\beta_1x_1+\beta_2x_2+\beta_3x_3+...$$

This approximation is good enough for 12 or more data points. Again, if you do this for just one or two groups, we’re already acquainted with those equations, i.e. the **Wilcoxon signed-rank test** or the **Mann-Whitney U test** respectively.

#### 6.1.2 Example data

We make a three-level factor with the levels `a`, `b`, and `c` so that the one-way ANOVA basically becomes a “three-sample t-test”. Then we manually do the dummy coding of the groups.

In [ ]:
# Three variables in "long" format
N = 20  # Number of samples per group
D = data.frame(
  value = c(rnorm_fixed(N, 0), rnorm_fixed(N, 1), rnorm_fixed(N, 0.5)),
  group = rep(c('a', 'b', 'c'), each = N),
  
  # Explicitly add indicator/dummy variables
  # Could also be done using model.matrix(~D$group)
  #group_a = rep(c(1, 0, 0), each=N),  # This is the intercept. No need to code
  group_b = rep(c(0, 1, 0), each = N),
  group_c = rep(c(0, 0, 1), each = N)
)  # N of each level
tail(D)

With group a’s intercept omni-present, see how exactly one other parameter is added to predict value for group b and c in a given row (`print(D)` to see). Thus data points in group b never affect the estimates in group c.

#### 6.1.3 R code: one-way ANOVA

OK, let’s see the identity between a dedicated ANOVA function (`car::Anova`) and the dummy-coded in-your-face linear model in `lm`.

In [ ]:
# Compare built-in and linear model
a = car::Anova(aov(value ~ group, D))  # Dedicated ANOVA function
b = lm(value ~ 1 + group_b + group_c, data = D)  # As in-your-face linear model

In [ ]:
print(a)

In [ ]:
print(coefficients(aov(value ~ group, D)))

In [ ]:
summary(b)

Actually, `car::Anova` and `aov` are wrappers around `lm` so the identity comes as no surprise. It only shows that the dummy-coded formula, which had a direct interpretation as a linear model, is the one that underlies the shorthand notation syntax `y ~ factor`. Indeed, the only real reason to use `aov` and `car::Anova` rather than `lm` is to get a nicely formatted ANOVA table.

The default output of lm returns parameter estimates as well (bonus!), which you can see if you unfold the R output above. However, because this IS the ANOVA model, you can also get parameter estimates out into the open by calling coefficients(aov(...)).

**Note** that I do not use the aov alone function because it computes type-I sum of squares, which is widely discouraged. There is a BIG polarized debate about whether to use type-II (as car::Anova does by default) or type-III sum of squares (set car::Anova(..., type=3)), but let’s skip that for now.

### 6.2 Two-way ANOVA
#### Theory: As linear models

Model: one mean per group (main effects) plus these means multiplied across factors (interaction effects). The main effects are the one-way ANOVAs above, though in the context of a larger model. The interaction effect is harder to explain in the abstract even though it’s just a few numbers multiplied with each other. I will leave that to the teachers to keep focus on equivalences here :-)

Switching to matrix notation:

$$ y = \beta_0 + \beta_1X_1 + \beta_2X_2 + \beta_3X_1X_2 $$ $$ H_0:\beta_3 = 0 $$

Here $\beta_i$ are vectors of betas of which only one is selected by the indicator vector $X_i$ . The $H_0$ shown here is the interaction effect. Note that the intercept $\beta_0$, to which all other $\beta$s are relative, is **now the mean for the first level of all factors.**

Continuing with the dataset from the one-way ANOVA above, let’s add a crossing factor mood so that we can test the `group:mood interaction` (a 3x2 ANOVA). We also do the dummy coding of this factor needed for the linear model.

In [ ]:
# Crossing factor
D$mood = c('happy', 'sad')

# Dummy coding
D$mood_happy = ifelse(D$mood == 'happy', 1, 0)  # 1 if mood==happy. 0 otherwise.

tail(D)

$\beta_0$ is now the happy guys from group a!

In [ ]:
means = lm(value ~ mood * group, D)$coefficients

P_anova2 = ggplot(D, aes(x=group, y=value, color=mood)) + 
  geom_segment(x = -10, xend = 100, y = means[1], yend = 0.5, col = 'blue', lwd = 2) +
  stat_summary(fun.y = mean, geom = "errorbar", aes(ymax = ..y.., ymin = ..y..),  lwd = 2)
theme_axis(P_anova2, xlim = c(-0.5, 3.5)) + theme(axis.text.x = element_text())

#### 6.2.2 R code: Two-way ANOVA

Now let’s turn to the actual modeling in R. We compare a dedicated ANOVA function (`car::Anova`; see One-Way ANOVA why) to the linear model (`lm`). Notice that in ANOVA, we are testing a full factor interaction all at once which involves many parameters (two in this case), so we can’t look at the overall model fit nor any particular parameter for the result. Therefore, I use a [likelihood-ratio test](https://en.wikipedia.org/wiki/Likelihood-ratio_test) to compare a full two-way ANOVA model (“saturated”) to one without the interaction effect(s). The anova function does this test. Even though that looks like cheating, it’s just computing likelihoods, p-values, etc. on the models that were already fitted, so it’s legit!

In [ ]:
# Dedicated two-way ANOVA functions
a = car::Anova(aov(value ~ mood * group, D), type='II')  # Normal notation. "*" both multiplies and adds main effects
b = car::Anova(aov(value ~ mood + group + mood:group, D))  # Identical but more verbose about main effects and interaction

# Testing the interaction terms as linear model.
full = lm(value ~ 1 + group_b + group_c + mood_happy + group_b:mood_happy + group_c:mood_happy, D)  # Full model
null = lm(value ~ 1 + group_b + group_c + mood_happy, D)  # Without interaction

# Likelihood-ratio test
c = anova(null, full)  # whoop whoop, same F, p, and Dfs

In [ ]:
print(a)

In [ ]:
print(c)

Below, I present approximate main effect models, though exact calculation of ANOVA main effects is [more involved](https://stats.idre.ucla.edu/stata/faq/how-can-i-get-anova-simple-main-effects-with-dummy-coding/) if it is to be accurate and furthermore depend on whether type-II or type-III sum of squares are used for inference.

Look at the model summary statistics to find values comparable to the Anova-estimated main effects above.

In [ ]:
# Main effect of group.
e = lm(value ~ 1 + group_b + group_c, D)

# Main effect of mood.
f = lm(value ~ 1 + mood_happy, D)

In [ ]:
summary(e)

In [ ]:
summary(f)

## 6.3 ANCOVA
This is simply ANOVA with a continuous regressor added so that it now contains continuous and (dummy-coded) categorical predictors. For example, if we continue with the one-way ANOVA example, we can add age and it is now called a one-way ANCOVA:

$$ y=\beta_0+\beta_1x_1+\beta_2x_2+...+\beta_3age $$

… where $x_i$ are our usual dummy-coded indicator variables. $\beta_0$ is now the mean for the first group at $age=0$. You can turn all ANOVAs into ANCOVAs this way, e.g. by adding $\beta_N⋅age$ to our two-way ANOVA in the previous section. But let us go ahead with our one-way ANCOVA, starting by adding $age$ to our dataset:

In [ ]:
# Update data with a continuous covariate
D$age = D$value + rnorm_fixed(nrow(D), sd = 3)  # Correlated to value
tail(D)

In [ ]:
# Dedicated ANCOVA functions. The order of factors matter in pure-aov (type-I variance).
# Use type-II (default for car::Anova) or type-III (set type=3),
a = car::Anova(aov(value ~ group + age, D))
#a = aov(value ~ group + age, D)  # Predictor order matters. Not nice!

# As dummy-coded linear model.
full = lm(value ~ 1 + group_b + group_c + age, D)

# Testing main effect of age using Likelihood-ratio test
null_age = lm(value ~ 1 + group_b + group_c, D)  # Full without age. One-way ANOVA!
result_age = anova(null_age, full)

# Testing main effect of groupusing Likelihood-ratio test
null_group = lm(value ~ 1 + age, D)  # Full without group. Pearson correlation!
result_group = anova(null_group, full)

In [ ]:
print(a)

In [ ]:
summary(full)

In [ ]:
print(result_age)

In [ ]:
print(result_group)

## 7 Proportions: Chi-square is a log-linear model

Recall that when you take the logarithm, you can easily make statements about *proportions*, i.e., that for every increase in $x$, $y$ increases a certain percentage. This turns out to be one of the simplest (and therefore best!) ways to make count data and contingency tables intelligible. See this [nice introduction](https://www.uni-tuebingen.de/fileadmin/Uni_Tuebingen/SFB/SFB_833/A_Bereich/A1/Christoph_Scheepers_-_Statistikworkshop.pdf) to Chi-Square tests as linear models.


### 7.1 Goodness of fit
#### 7.1.1 Theory: As log-linear model
Model: a single intercept predicts $log(y)$. I’ll refer you to take a look at the section on contingency tables which is basically a “two-way goodness of fit”.

#### 7.1.2 Example data

For this, we need some wide count data:

In [ ]:
# Data in long format
D = data.frame(mood = c('happy', 'sad', 'meh'),
               counts = c(60, 90, 70))

# Dummy coding for the linear model
D$mood_happy = ifelse(D$mood == 'happy', 1, 0)
D$mood_sad = ifelse(D$mood == 'sad', 1, 0)
D

#### 7.1.3 R code: Goodness of fit

Now let’s see that the Goodness of fit is just a log-linear equivalent to a one-way ANOVA. We set `family = poisson()` which defaults to setting a logarithmic [link function](https://en.wikipedia.org/wiki/Generalized_linear_model#Link_function) (`family = poisson(link='log')`).

In [ ]:
# Built-in test
a = chisq.test(D$counts)

# As log-linear model, comparing to an intercept-only model
full = glm(counts ~ 1 + mood_happy + mood_sad, data = D, family = poisson())
null = glm(counts ~ 1, data = D, family = poisson())
b = anova(null, full, test = 'Rao')

# Note: glm can also do the dummy coding for you:
c = glm(counts ~ mood, data = D, family = poisson())

In [ ]:
print(a)

In [ ]:
print(b)

### 7.2 Contingency tables
#### 7.2.1 Theory: As log-linear model

The theory here will be a bit more convoluted, and I mainly write it up so that you can get the feeling that it really is just a log-linear two-way ANOVA model. Let’s get started…

For a two-way contingency table, the model of the count variable $y$ is a modeled using the marginal proportions of a contingency table. Why this makes sense, is too involved to go into here, but see the [relevant slides](https://uni-tuebingen.de/fileadmin/Uni_Tuebingen/SFB/SFB_833/A_Bereich/A1/Christoph_Scheepers_-_Statistikworkshop.pdf) by Christoph Scheepers here for an excellent exposition. The model is composed of a lot of counts and the regression coefficients $A_i$ and $B_i$:

$$ y_i=N⋅x_i(A_i/N)⋅z_j(B_j/N)⋅x_ij/((A_ix_i)/(B_jz_j)/N) $$

What a mess!!! Here, $i$ is the row index, $j$ is the column index, $x_x$
 is the sum of that row and/or column, $N=sum(y)$. Remember that $y$ is a count variable, so $N$ is just the total count.

We can simplify the notation by defining the *proportions:* $\alpha_i=x_i(A_i/N)$, $\beta_i=x_j(B_i/N)$ and $\alpha_i\beta_i=x_ij/(A_ix_i)/(B_jz_j)/N$
. Let’s write the model again:

$$ y_i=N⋅\alpha_i⋅\beta_j⋅\alpha_i\beta_j $$

Ah, much prettier. However, there is still lot’s of multiplication which makes it hard to get an intuition about how the actual numbers interact. We can make it much more intelligible when we remember that $log(A⋅B)=log(A)+log(B)$. Doing logarithms on both sides, we get:

$$log(y_i)=log(N)+log(\alpha_i)+log(beta_j)+log(\alpha_ibeta_j)$$

Snuggly! Now we can get a better grasp on how the regression coefficients (which are proportions) independently contribute to y. This is why logarithms are so nice for proportions. Note that this is just the two-way ANOVA model with some logarithms added, so we are back to our good old linear models - only the interpretation of the regression coefficients have changed! And we cannot use lm anymore in R.

#### 7.2.2 Example data

Here we need some long data and we need it in table format for `chisq.test`:

In [ ]:
# Contingency data in long format for linear model
D = data.frame(
  mood = c('happy', 'happy', 'meh', 'meh', 'sad', 'sad'),
  sex = c('male', 'female', 'male', 'female', 'male', 'female'),
  Freq = c(100, 70, 30, 32, 110, 120)
)

# ... and as table for chisq.test
D_table = D %>%
  spread(key = mood, value = Freq) %>%  # Mood to columns
  select(-sex) %>%  # Remove sex column
  as.matrix()

# Dummy coding of D for linear model (skipping mood=="sad" and gender=="female")
# We could also use model.matrix(D$Freq~D$mood*D$sex)
D$mood_happy = ifelse(D$mood == 'happy', 1, 0)
D$mood_meh = ifelse(D$mood == 'meh', 1, 0)
D$sex_male = ifelse(D$sex == 'male', 1, 0)
D

#### 7.2.3 R code: Chi-square test

Now let’s show the equivalence between a chi-square model and a log-linear model. This is very similar to our two-way ANOVA above:

In [ ]:
# Built-in chi-square. It requires matrix format.
a = chisq.test(D_table)

# Using glm to do a log-linear model, we get identical results when testing the interaction term:
full = glm(Freq ~ 1 + mood_happy + mood_meh + sex_male + mood_happy*sex_male + mood_meh*sex_male, data = D, family = poisson())
null = glm(Freq ~ 1 + mood_happy + mood_meh + sex_male, data = D, family = poisson())
b = anova(null, full, test = 'Rao')  # Could also use test='LRT' or test='Chisq'

# Note: let glm do the dummy coding for you
full = glm(Freq ~ mood * sex, family = poisson(), data = D)
c = anova(full, test = 'Rao')

# Note: even simpler syntax using MASS:loglm ("log-linear model")
d = MASS::loglm(Freq ~ mood + sex, D)

In [ ]:
print(a)

In [ ]:
print(b)

In [ ]:
summary(full)

If you unfold the raw R output, I’ve included `summary(full)` so that you can see the raw regression coefficients. Being a log-linear model, these are the percentage increase in $y$ over and above the intercept if that category obtains.

## 9 Limitations
I have made a few simplifications for clarity:

1- I have not covered assumptions in the examples. This will be another post! But all assumptions of all tests come down to the usual three: a) independence of data points, b) normally distributed residuals, and c) homoscedasticity.

2- I assume that all null hypotheses are the absence of an effect, but everything works the same for non-zero null hypotheses.

3- I have not discussed inference. I am only including p-values in the comparisons as a crude way to show the equivalences between the underlying models since people care about p-values. Parameter estimates will show the same equivalence. How to do inference is another matter. Personally, I’m a Bayesian, but going Bayesian here would render it less accessible to the wider audience. Also, doing robust models would be preferable, but fail to show the equivalences.